In [182]:
import warnings
from statsmodels.tools.sm_exceptions import (
    PerfectSeparationWarning, 
    HessianInversionWarning
)

# Suppress all relevant warnings
warnings.filterwarnings('ignore', category=PerfectSeparationWarning)
warnings.filterwarnings('ignore', category=HessianInversionWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import pandas as pd
import statsmodels.api as sm
import numpy as np
import pickle

# Function to create directories
def create_trial_directory(dir_name):
    dir_path = os.path.join(os.getcwd(), dir_name)
    os.makedirs(dir_path, exist_ok=True)
    return dir_path

# Creating directories
trial_pp_dir = create_trial_directory("trial_pp")
trial_itt_dir = create_trial_directory("trial_itt")

# Load dataset
data_censored = pd.read_csv('c:/Users/USER/Documents/3rd year 2nd sem/Data Analytics/Assignments_Data_Analytics/Assignment_1_Clustering_Data_Analytics/data_censored.csv')

# Define logistic model fitting function with improved stability
def fit_logistic_model(data, predictors, outcome):
    X = sm.add_constant(data[predictors])
    y = data[outcome]
    try:
        model = sm.Logit(y, X).fit(disp=0, method="lbfgs")
    except Exception as e:
        print(f"Warning: Logistic model fitting failed for {outcome} with error: {e}")
        return None
    return model

class TrialSequence:
    def __init__(self, estimand, data=None):
        self.estimand = estimand
        self.data = data
        self.switch_weights = None
        self.censor_weight_model = None
        self.weights = {}         # For informative censoring models
        self.switch_models = {}   # For treatment switching models

    def set_data(self, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.data = data[[id_col, period_col, treatment_col, outcome_col, eligible_col,
                          'x1', 'x2', 'x3', 'x4', 'age', 'age_s', 'censored']].copy()

    def set_switch_weight_model(self, numerator_predictors, denominator_predictors):
        # This method is only applicable for PP estimand.
        if self.estimand != "PP":
            print("Switch weight model is only applicable to PP estimand.")
            return

        data = self.data.copy()

        # Split data by previous treatment for treatment switching models.
        data_t1 = data[data['treatment'] == 1].copy()  # previous treatment = 1
        data_t0 = data[data['treatment'] == 0].copy()  # previous treatment = 0

        # Fit numerator models
        num_model_t1 = fit_logistic_model(data_t1, numerator_predictors, outcome='treatment')
        num_model_t0 = fit_logistic_model(data_t0, numerator_predictors, outcome='treatment')

        # Fit denominator models
        denom_model_t1 = fit_logistic_model(data_t1, denominator_predictors, outcome='treatment')
        denom_model_t0 = fit_logistic_model(data_t0, denominator_predictors, outcome='treatment')

        if any(m is None for m in [num_model_t1, num_model_t0, denom_model_t1, denom_model_t0]):
            print("Error: One or more switching models failed to fit. Skipping switch weight computation.")
            return

        # Save the switching models
        self.switch_models['numerator_n1'] = num_model_t1
        self.switch_models['numerator_n0'] = num_model_t0
        self.switch_models['denominator_d1'] = denom_model_t1
        self.switch_models['denominator_d0'] = denom_model_t0

        # Compute predicted probabilities separately for each group
        data['num_prob'] = 0
        data['denom_prob'] = 0
        for grp in [1, 0]:
            subset = data[data['treatment'] == grp]
            if grp == 1:
                X_num = sm.add_constant(subset[numerator_predictors])
                X_denom = sm.add_constant(subset[denominator_predictors])
                data.loc[subset.index, 'num_prob'] = num_model_t1.predict(X_num).astype(float)
                data.loc[subset.index, 'denom_prob'] = denom_model_t1.predict(X_denom).astype(float)
            else:
                X_num = sm.add_constant(subset[numerator_predictors])
                X_denom = sm.add_constant(subset[denominator_predictors])
                data.loc[subset.index, 'num_prob'] = num_model_t0.predict(X_num).astype(float)
                data.loc[subset.index, 'denom_prob'] = denom_model_t0.predict(X_denom).astype(float)

        # Avoid division by zero
        data['denom_prob'] = np.clip(data['denom_prob'], 1e-6, 1)

        data['switch_weight'] = data['num_prob'] / data['denom_prob']
        data['switch_weight'] = np.clip(data['switch_weight'], 0.01, 10)

        self.switch_weights = data[["id", "period", "treatment", "switch_weight"]]
        print("Switch weights successfully calculated.")

    def set_censor_weight_model(self, censor_event, numerator, denominator):
        self.censor_weight_model = {'censor_event': censor_event, 'numerator': numerator, 'denominator': denominator}

    def calculate_weights(self):
        if self.censor_weight_model:
            self.fit_censoring_models()
        print(f"Weight models calculated for {self.estimand}.")

        if self.data is None:
            print("Error: No data available. Please set data using set_data().")
            return

        print(f"Calculating final weights for {self.estimand}...")

        # Step 1: Apply switch weights (Treatment Switching)
        if self.switch_weights is not None:
            self.data = self.data.merge(self.switch_weights, on=["id", "period", "treatment"], how="left")
        else:
            print("Warning: Switch weights not set. Using default of 1.")
            self.data["switch_weight"] = 1

        # Step 2: Compute Censoring Weights (Inverse Probability of Remaining Uncensored)
        self.data["not_censored"] = 1 - self.data["censored"]

        # For censoring, compute predicted probabilities for numerator and denominator by treatment
        if self.estimand == "PP":
            # For PP, use separate models for treatment 0 and 1
            for grp in [0, 1]:
                idx = self.data[self.data["treatment"] == grp].index
                num_model = self.weights.get(f"numerator_n{grp}")
                den_model = self.weights.get(f"denominator_d{grp}")
                if num_model is not None and den_model is not None:
                    X_num = sm.add_constant(self.data.loc[idx, ['x2']])
                    X_den = sm.add_constant(self.data.loc[idx, ['x2', 'x1']])
                    self.data.loc[idx, "numerator"] = num_model.predict(X_num).astype(float)
                    self.data.loc[idx, "denominator"] = den_model.predict(X_den).astype(float)
                else:
                    self.data.loc[idx, "numerator"] = 1
                    self.data.loc[idx, "denominator"] = 1
        else:
            # ITT: Use the overall numerator model (if available) and group‐wise denominator models
            idx_all = self.data.index
            num_model = self.weights.get("numerator")
            if num_model is not None:
                X_num = sm.add_constant(self.data.loc[idx_all, ['x2']])
                self.data.loc[idx_all, "numerator"] = num_model.predict(X_num).astype(float)
            else:
                self.data.loc[idx_all, "numerator"] = 1
            for grp in [0, 1]:
                idx = self.data[self.data["treatment"] == grp].index
                den_model = self.weights.get(f"denominator_d{grp}")
                if den_model is not None:
                    X_den = sm.add_constant(self.data.loc[idx, ['x2', 'x1']])
                    self.data.loc[idx, "denominator"] = den_model.predict(X_den).astype(float)
                else:
                    self.data.loc[idx, "denominator"] = 1

        # Avoid division by zero and compute censoring weight
        self.data["denominator"] = np.clip(self.data["denominator"], 1e-6, 1)
        self.data["censor_weight"] = self.data["numerator"] / self.data["denominator"]
        self.data["censor_weight"] = np.clip(self.data["censor_weight"], 0.01, 10)

        # Step 3: Compute Final Stabilized Weights
        self.data["final_weight"] = self.data["switch_weight"] * self.data["censor_weight"]
        print("Final weights computed successfully.")

    def fit_censoring_models(self):
        self.data["not_censored"] = 1 - self.data["censored"]
        if self.estimand == "PP":
            data_n0 = self.data[(self.data['treatment'] == 0) & (self.data['eligible'] == 1)].copy()
            data_n1 = self.data[(self.data['treatment'] == 1) & (self.data['eligible'] == 1)].copy()
            self.weights['numerator_n0'] = fit_logistic_model(data_n0, ['x2'], outcome='not_censored')
            self.weights['numerator_n1'] = fit_logistic_model(data_n1, ['x2'], outcome='not_censored')
        else:
            self.weights['numerator'] = fit_logistic_model(self.data, ['x2'], outcome='not_censored')
        data_d0 = self.data[self.data['treatment'] == 0].copy()
        data_d1 = self.data[self.data['treatment'] == 1].copy()
        self.weights['denominator_d0'] = fit_logistic_model(data_d0, ['x2', 'x1'], outcome='not_censored')
        self.weights['denominator_d1'] = fit_logistic_model(data_d1, ['x2', 'x1'], outcome='not_censored')

# Function to display and save weight model summaries in the trialemulation style
def show_weight_models(trial):
    # Determine readable header based on estimand type
    if trial.estimand == "ITT":
        header = "Weight Models for Informative Censoring (ITT)"
    elif trial.estimand == "PP":
        header = "Weight Models for Informative Censoring (PP)"
    else:
        header = "Weight Models for Informative Censoring"
        
    print(header)
    print("-" * len(header) + "\n")
    
    for key, model in trial.weights.items():
        if model is None:
            print(f"Warning: Model for key '{key}' is not available.\n")
            continue

        # For PP, label using the group indicator (e.g., n0, n1, d0, d1)
        if trial.estimand == "PP":
            if key.startswith("numerator_n"):
                label = "n" + key.split("_")[-1]
            elif key.startswith("denominator_d"):
                label = "d" + key.split("_")[-1]
            else:
                label = key
        else:
            # For ITT, label numerator as 'n' and denominator as 'd{grp}'
            if key == "numerator":
                label = "n"
            elif key.startswith("denominator_d"):
                label = "d" + key.split("_")[-1]
            else:
                label = key

        print(f"[[{label}]]")
        try:
            summary_text = model.summary2().as_text()
        except Exception:
            summary_text = model.summary().as_text()
        print(summary_text + "\n")

        # Save the model object to disk in a subfolder "switch_models"
        dir_path = trial_pp_dir if trial.estimand == "PP" else trial_itt_dir
        model_dir = os.path.join(dir_path, "switch_models")
        os.makedirs(model_dir, exist_ok=True)
        model_filename = f"model_{key}.pkl"
        model_path = os.path.join(model_dir, model_filename)
        with open(model_path, "wb") as f:
            pickle.dump(model, f)
        print("path")
        print(model_path + "\n")
    
    # For PP, also display Treatment Switching Models
    if trial.estimand == "PP" and trial.switch_models:
        header_ts = "Weight Models for Treatment Switching (PP)"
        print(header_ts)
        print("-" * len(header_ts) + "\n")
        for key, model in trial.switch_models.items():
            if model is None:
                print(f"Warning: Model for key '{key}' is not available.\n")
                continue
            label = key.split("_")[-1]  # e.g. n0, n1, d0, d1
            print(f"[[{label}]]")
            try:
                summary_text = model.summary2().as_text()
            except Exception:
                summary_text = model.summary().as_text()
            print(summary_text + "\n")

            # Save the switching model object to disk in a separate folder.
            dir_path = trial_pp_dir if trial.estimand == "PP" else trial_itt_dir
            model_dir = os.path.join(dir_path, "switching_models")
            os.makedirs(model_dir, exist_ok=True)
            model_filename = f"model_{key}.pkl"
            model_path = os.path.join(model_dir, model_filename)
            with open(model_path, "wb") as f:
                pickle.dump(model, f)
            print("path")
            print(model_path + "\n")

# Process ITT trial
trial_itt = TrialSequence("ITT")
trial_itt.set_data(data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_itt.set_censor_weight_model("censored", ["x2"], ["x2", "x1"])
trial_itt.calculate_weights()

# Display weight model summaries for ITT trial
show_weight_models(trial_itt)

# Process PP trial
trial_pp = TrialSequence("PP")
trial_pp.set_data(data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_pp.set_switch_weight_model(["age"], ["age", "x1", "x3"])
trial_pp.set_censor_weight_model("censored", ["x2"], ["x2", "x1"])
trial_pp.calculate_weights()

# Display weight model summaries for PP trial
show_weight_models(trial_pp)

Weight models calculated for ITT.
Calculating final weights for ITT...
Final weights computed successfully.
Weight Models for Informative Censoring (ITT)
---------------------------------------------

[[n]]
                         Results: Logit
Model:              Logit            Method:           MLE      
Dependent Variable: not_censored     Pseudo R-squared: 0.027    
Date:               2025-03-02 19:19 AIC:              397.4004 
No. Observations:   725              BIC:              406.5727 
Df Model:           1                Log-Likelihood:   -196.70  
Df Residuals:       723              LL-Null:          -202.11  
Converged:          1.0000           LLR p-value:      0.0010067
No. Iterations:     7.0000           Scale:            1.0000   
------------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
------------------------------------------------------------------
const     2.4481     0.1406   17